In [1]:

!pip install black[jupyter] --quiet
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/DL_Final

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/MyDrive/DL_Final


In [2]:
!black /content/drive/MyDrive/DL_Final/'Training.ipynb'

reformatted /content/drive/MyDrive/DL_Final/Training.ipynb

All done! ✨ 🍰 ✨
1 file reformatted.


In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import make_grid
import torchvision.transforms as transforms

import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torchvision.models import resnet50
from scipy.spatial.distance import cosine
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:



# Define a function to create data loaders for training and testing
def create_data_loaders(conf):
    # Define the transformation
    transform = transforms.Compose(
        [
            transforms.Resize((conf.img_size, conf.img_size)),  # Resize the image
            transforms.ToTensor(),  # Convert the image to a tensor
            transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),  # Normalize the tensor
        ]
    )

    # Create the training and testing datasets
    train_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=conf.batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=conf.batch_size, shuffle=False)

    return train_loader, test_loader

class Config:
    img_size = 32
    batch_size = 64


conf = Config()
data_loader, test_loader = create_data_loaders(conf)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:

import os
import models
from models import Generator, WGAN_Discriminator

# Create directory for checkpoints
checkpoints_dir = './checkpoints'
os.makedirs(checkpoints_dir, exist_ok=True)

# Initialize the WGAN generator and discriminator
generator = Generator(noise_dim=100, num_classes=10)
wgan_discriminator = WGAN_Discriminator(num_classes=10)

# RMSprop optimizer for stability
optimizer_G = optim.RMSprop(generator.parameters(), lr=0.00005)
optimizer_D = optim.RMSprop(wgan_discriminator.parameters(), lr=0.00005)

# Training parameters
num_epochs = 20
n_critic = 5
clip_value = 0.01

# Record loss values during training
losses_D = []
losses_G = []

# WGAN training loop with checkpoint saving
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        batch_size = images.size(0)

        # Discriminator training
        for _ in range(n_critic):
            optimizer_D.zero_grad()

            real_outputs = wgan_discriminator(images, labels)
            noise = torch.randn(batch_size, 100)
            fake_images = generator(noise, labels)
            fake_outputs = wgan_discriminator(fake_images.detach(), labels)

            loss_D = fake_outputs.mean() - real_outputs.mean()
            loss_D.backward()
            optimizer_D.step()

            # Weight clipping for stability
            for param in wgan_discriminator.parameters():
                param.data.clamp_(-clip_value, clip_value)

            losses_D.append(loss_D.item())

        # Generator training
        optimizer_G.zero_grad()
        fake_outputs = wgan_discriminator(fake_images, labels)
        loss_G = -fake_outputs.mean()
        loss_G.backward()
        optimizer_G.step()

        losses_G.append(loss_G.item())

        # Save complete models every 100 iterations
        if i % 50 == 0:
            torch.save(generator, f"{checkpoints_dir}/generator_epoch{epoch}_step{i}.pth")
            torch.save(wgan_discriminator, f"{checkpoints_dir}/discriminator_epoch{epoch}_step{i}.pth")

            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(data_loader)}], Loss D: {loss_D.item()}, Loss G: {loss_G.item()}")


Epoch [0/20], Step [0/782], Loss D: -0.04757065698504448, Loss G: 0.04148619994521141
Epoch [0/20], Step [50/782], Loss D: -0.9229906797409058, Loss G: 0.48850375413894653
Epoch [0/20], Step [100/782], Loss D: -1.0120543241500854, Loss G: 0.41942673921585083
Epoch [0/20], Step [150/782], Loss D: -0.7935128211975098, Loss G: 0.38479748368263245
Epoch [0/20], Step [200/782], Loss D: -0.9455289840698242, Loss G: 0.5102406740188599
Epoch [0/20], Step [250/782], Loss D: -0.33532804250717163, Loss G: 0.2405933439731598
